In [2]:
import ROOT
import numpy as np
from src.histogram import Histogram1D

ROOT.TH1.SetDefaultSumw2()  # Sets weighted binning in all ROOT histograms by default
# ROOT.TH1.AddDirectory(False)
# ROOT.gROOT.SetBatch()  # Prevents TCanvas popups
np.random.seed(42)
c = ROOT.TCanvas()

a = np.random.normal(0, 1, 1000)
b = np.random.normal(0, 1, 1000)
# b = [-.9, -.7, -.5, -.3, -.1, .1, .3, .5, .7, .9]
w = np.random.random(1000)
w2 = np.random.random(1000)
# w2 = np.ones(20)
nbins = 10
xmin = -1
xmax = 1

In [3]:
hH = Histogram1D(a, bins=(nbins, xmin, xmax), weight=w)
h2 = Histogram1D(b, bins=(nbins, xmin, xmax), weight=w2)

hH_div = hH / h2

hH_remul = hH_div * h2

print(hH_div)
print('original mean:     ', hH.mean)
print('second hist mean:  ', h2.mean)
print('divided mean:      ', hH_div.mean)
print('remultiplied mean: ', hH_remul.mean)

                    ┌────────────────────────────────────────────────────────┐
[-inf,   -1) 1.075  │████████████████████████████████▋                       │
[  -1, -0.8) 1.192  │████████████████████████████████████▎                   │
[-0.8, -0.6) 0.897  │███████████████████████████▎                            │
[-0.6, -0.4) 1.811  │███████████████████████████████████████████████████████ │
[-0.4, -0.2) 0.7816 │███████████████████████▊                                │
[-0.2,    0) 0.8155 │████████████████████████▊                               │
[   0,  0.2) 1.145  │██████████████████████████████████▊                     │
[ 0.2,  0.4) 1.381  │█████████████████████████████████████████▉              │
[ 0.4,  0.6) 0.8047 │████████████████████████▍                               │
[ 0.6,  0.8) 1.251  │██████████████████████████████████████                  │
[ 0.8,    1) 0.8649 │██████████████████████████▎                             │
[   1,  inf) 0.8871 │██████████████████████████▉    

In [4]:
hR = ROOT.TH1F('test', 'test;x;entries', nbins, xmin, xmax)
hR2 = ROOT.TH1F('test2', 'test;x;entries', nbins, xmin, xmax)
for ia, ib, iw, iw2 in zip(a, b, w, w2):
	hR.Fill(ia, iw)
	hR2.Fill(ib, iw2)

hR_div = hR.Clone()
hR_div.Divide(hR2)

hR_remul = hR_div.Clone()
hR_remul.Multiply(hR2)

print('original mean:     ', hR.GetMean())
print('second hist mean:  ', hR2.GetMean())
print('divided mean:      ', hR_div.GetMean())
print('remultiplied mean: ', hR_remul.GetMean())

original mean:      -0.0024607456905039724
second hist mean:   0.015514270583447837
divided mean:       -0.03079575110606322
remultiplied mean:  -0.0034098056425223688


In [32]:
hR.SetBinContent(1, None)
hR.SetBinContent(2, None)
b = hR.Fit('pol0', "QS")
print(b)
print("Chi2: ", b.Chi2())
print("error: ", b.Error(0))
print("NDf: ", b.Ndf())
print("params: ", b.Parameters()[0])
c.Draw()

TypeError: none of the 3 overloaded methods succeeded. Full details:
  void TH1::SetBinContent(int bin, double content) =>
    TypeError: could not convert argument 2 (must be real number, not NoneType)
  void TH1::SetBinContent(int bin, int, double content) =>
    TypeError: takes at least 3 arguments (2 given)
  void TH1::SetBinContent(int bin, int, int, double content) =>
    TypeError: takes at least 4 arguments (2 given)

In [16]:
th_2bin_low = ROOT.TH1F('2 bin low' , '2 bin low', 2, 0., 2.)
th_2bin_low.Fill(0.1, 0.5)
th_2bin_low.Fill(1.1, 2)
print(th_2bin_low.GetMean())

0.9


In [17]:
th_2bin_hi = ROOT.TH1F('2 bin hi' , '2 bin hi', 2, 0., 2.)
th_2bin_hi.Fill(0.9, 0.5)
th_2bin_hi.Fill(1.9, 2.)
print(th_2bin_hi.GetMean())

1.7


In [18]:
bh_2bin_low = Histogram1D([0.1, 1.1], (2, 0., 2.), weight=[0.5, 2])
print(bh_2bin_low)
print(bh_2bin_low.mean)
bh_2bin_hi = Histogram1D([0.9, 1.9], (2, 0., 2.), weight=[0.5, 2])
print(bh_2bin_hi)
print(bh_2bin_hi.mean)

                ┌────────────────────────────────────────────────────────────┐
[-inf,   0) 0   │                                                            │
[   0,   1) 0.5 │██████████████▊                                             │
[   1,   2) 2   │███████████████████████████████████████████████████████████ │
[   2, inf) 0   │                                                            │
                └────────────────────────────────────────────────────────────┘
1.3
                ┌────────────────────────────────────────────────────────────┐
[-inf,   0) 0   │                                                            │
[   0,   1) 0.5 │██████████████▊                                             │
[   1,   2) 2   │███████████████████████████████████████████████████████████ │
[   2, inf) 0   │                                                            │
                └────────────────────────────────────────────────────────────┘
1.3
